In [341]:
# %store -r tokens
import pickleshare

# Connect to the PickleShare database
db = pickleshare.PickleShareDB('~/.ipython/profile_default/db')

# Retrieve variables
tokens = db['tokens']
char_lit=db['char_lit']
const_str=db['const_str']


In [342]:
tokens = tokens

In [343]:
global_symbol_table = {}

# symbol table structure {'a':['int'], 'func1' : ['int', {'a': 'int'}], 'func2' : ['void', {}]}

### Ignore Preprocessors

In [344]:
def ignorePreprocessor(tokens, index):
    
    if(index<len(tokens) and tokens[index][0] != '#'):
        return index, "not an pre processor"
    
    index += 1
    
    if(index<len(tokens) and tokens[index] != '<'):
        return index, "invalid pre processor"
    index += 1
    
    while(index<len(tokens) and tokens[index] != '>'):
        index += 1

    if(index<len(tokens) and tokens[index] != '>'):
        return index, "invalid operator"
 
    return index+1, None

In [345]:
def ignoreTillSemiColon(tokens, index):
    
    while(index<len(tokens) and tokens[index] != ';'):
        index += 1

    return index+1, None


In [346]:
datatype=["int","double","float","char","string", "void", "bool"]

integer_dtypes = ["int"]
decimal_dtypes = ["float", "double"]
boolean_dtypes = ["bool"]

In [347]:
rval_operators=['+', '-', '*', '/', '&&', '||', '&', '|', '<', '>', '^', '==', '!=' '>=', '<=']
lval_operators=['+=', '-=', '=', '*=', '/=', '|=', '&=']
lval_unary_operators=['++', '--']
unary=['!', '-', '+']

# Check identifier and constants

In [348]:
# returns isidentifier
def isIdentifier(s):
    if(not((s[0]=='_')or(s[0]>='a' and s[0]<='z')or(s[0]>='A' and s[0]<='Z'))):
        return False
    for i in range(1,len(s)):
        if(not((s[i]=='_')or(s[i]>='a' and s[i]<='z')or(s[i]>='A' and s[i]<='Z')or(s[i]>='0' and s[i]<='9'))):
            return False
    return True

# return isConst, dtype
def isConst(s):
    dtype = ""

    if(s == "true" or s == "false"):
        return True, "bool"
    
    if(s[0] == '"' and s[-1] == '"'):
        return True, "string"
    
    if(s[0] == "'" and s[-1] == "'" and len(s) == 3):
        return True, "char"

    try:
        int(s)
        return True, "int"
    except ValueError:
        try:
            float(s)
            return True, "float"
        except ValueError:
            return False, dtype


# CheckLval

In [349]:
# returns index, err, dtype, islval

def checkLval(tokens, index, symbol_table):
    
    
    dtype = ""
    if(index>=len(tokens)):
        return index, "Expression invalid",dtype,False
    res, const_dtype = isConst(tokens[index])
    if(res):
        index += 1
        return index, None, const_dtype, False
    
    if(index<len(tokens) and isIdentifier(tokens[index])):
        # check if present in symbol table and get dtype from symbol table
        ident = tokens[index]

        if(ident not in symbol_table):
            return index, "variable not declared", dtype, False
        index += 1

        # if the ident is a function
        if(index<len(tokens) and tokens[index] == '('):
            index += 1
            params = []
            

            if(index<len(tokens) and tokens[index] != ')'):
                

                while True:
                    if(index<len(tokens) and not isIdentifier(tokens[index])):
                        return index, "invalid identifier", dtype, False
                    
                    id = tokens[index]
                    
                    index+=1
                    if(id in symbol_table):
                        params.append(symbol_table[id][0])
                    else:
                        return index, "identifier not defined", dtype, False
                    if(index<len(tokens) and tokens[index] != ','):
                        break
                    index += 1
                
                
                if(len(symbol_table[ident]) < 2):
                    return index, "identifier is not callable", dtype, False
                
                # check this after existing from while loop and take care of , as well
                
                params_list = list(p[0] for p in symbol_table[ident][1].values())
                
                if(params_list != params):
                    return index, "parameter list of function doesn't match", dtype, False
                
                if(index<len(tokens) and tokens[index] != ')'):
                    return index, "missing closing parenthesis"
                index += 1
            else:
                if(len(symbol_table[ident]) < 2):
                    return index, "identifier is not callable", dtype, False

                params_list = list(symbol_table[ident][1].values())
                if(params_list != params):
                    return index, "parameter list of function doesn't match", dtype, False

        if(index<len(tokens) and tokens[index] in lval_unary_operators):
            if(len(symbol_table[ident]) >= 2):
                return index, "function identifier cannot be incremented", dtype, False
            if(symbol_table[ident][0] == "string"):
                return index, "string cannot be incremented", dtype, False
            index += 1
            
   
        # if not a function call
        return index, None, symbol_table[ident][0], True
        
    if(index<len(tokens) and tokens[index] in unary):
        op = tokens[index]
        index += 1
        index, err, data_type, sub_lval = checkLval(tokens=tokens, index=index, symbol_table=symbol_table)
        
        if(op == '!'):
            return index, err, 'bool', False
        else: # operator = '-' or '+'
            if((data_type not in integer_dtypes) and (data_type not in decimal_dtypes) and (data_type not in boolean_dtypes)):
                return index, f"operator {op} on non numerical data type", dtype, False
            return index, err, data_type, False
    
    if(index<len(tokens) and tokens[index] in lval_unary_operators):
        op = tokens[index]
        index += 1
        index, err, data_type, sub_lval = checkLval(tokens=tokens, index=index, symbol_table=symbol_table)
        if(not sub_lval):
            return index, f"{tokens[index]} allowed only for lvalue", data_type, False
        if(data_type == "string"):
            return index, "operator not allowed for string", data_type, False
        
        return index, None, data_type, True

    if(index<len(tokens) and tokens[index] == '('):
        index += 1
        index, err, data_type = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table, islval=False)
        if(err):
            return index, err, data_type, False
        
        if(index<len(tokens) and tokens[index] != ')'):
            return index, "unmatched braces", data_type, False
        index += 1 # matched braces
        return index, err, data_type, False

    return index, "unexpected token inside lvalue", dtype, False


# CheckExpr

In [350]:
# It doesn't parses semicolon
# returns index, err, dtype

def checkExpr(tokens, index, symbol_table, islval=True):
    
    dtype = ""
    index, err, ldtype, islvalue = checkLval(tokens, index, symbol_table)
    rdtype = ""
    
    if(err):
        return index, err, dtype

    if(index<len(tokens) and tokens[index] in lval_operators):
        if(islval and islvalue):
            index += 1
            index, err, rdtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table, islval=True)
            
        else:
            return index, "lval operators cannot be applied on rval", dtype
    
    elif(index<len(tokens) and tokens[index] in rval_operators):
        index += 1
        index, err, rdtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table, islval=False)
        

    else:
        return index, None, ldtype # if there is no operator after an lvalue

    if(err):
        return index, err, dtype
    
    if(index>=len(tokens)):
        return index, "expression declared incorrectly",dtype
    # decide dtype on basis of ldtype and rdtype
    if(ldtype == "void" or rdtype == "void"):
        
        return index, "no operation allowed on void datatype", dtype

    if(ldtype == "string"):
        
        if(not (rdtype == "char" or rdtype == "string")):
            return index, "mismatched types", dtype
        return index, None, "string"
    
    if(ldtype == "char"):
        
        if(rdtype == "string"):
            return index, "string cannot be appended to a char", dtype
        
        if(rdtype in boolean_dtypes):
            return index, None, "int"

        return index, None, rdtype

    if(ldtype in integer_dtypes):
        
        if(rdtype == "string"):
            return index, "mismatched types", dtype

        if(rdtype == "char"):
            return index, None, ldtype

        if(rdtype in boolean_dtypes):
            return index, None, ldtype

        if(rdtype in decimal_dtypes):
            return index, None, rdtype

        if(rdtype in integer_dtypes):
            if(integer_dtypes.index(rdtype) >= integer_dtypes.index(ldtype)):
                return index,None, rdtype
            else:
                return index,None, ldtype

    if(ldtype in decimal_dtypes):
        
        if(rdtype == "string"):
            return index, "mismatched types", dtype

        if(rdtype == "char"):
            return index, None, ldtype

        if(rdtype in boolean_dtypes):
            return index, None, ldtype

        if(rdtype in integer_dtypes):
            return index, None, ldtype

        if(rdtype in decimal_dtypes):
            if(decimal_dtypes.index(rdtype) >= decimal_dtypes.index(ldtype)):
                return index,None, rdtype
            else:
                return index,None, ldtype

    if(ldtype in boolean_dtypes):
        
        if(rdtype == "string"):
            return index, "mismatched types", dtype

        if(rdtype == "char"):
            return index, None, "int"
        
        return index, None, rdtype
    
    return index, "datatype invalid", dtype


# Check function declaration and variable definition ( Scoped )

In [351]:
# returns index, err, symbol_table
def checkFuncDecOrValDef(tokens, index, symbol_table):
    if(index<len(tokens) and tokens[index] in datatype):
        index, err, dtype, ident = checkVarHeader(tokens=tokens, index=index)

    if(err):
        return index, err, symbol_table

    symbol_table[ident] = [dtype]

    
    # variable decl
    if(index<len(tokens) and tokens[index] == ";"):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation", symbol_table
        return index+1, None, symbol_table
    
        # variable def
    if(index<len(tokens) and tokens[index] == "="):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation", symbol_table
        index += 1
        index,err,expr_dtype =  checkExpr(tokens, index, symbol_table)
        
        if(err):
            return index, err, symbol_table
        if(index<len(tokens) and tokens[index] == ","):
            while(index<len(tokens) and tokens[index] == ','):
                index += 1
                if(index<len(tokens) and not isIdentifier(tokens[index])):
                    return index, "expected identifier"
                symbol_table[tokens[index]] = [dtype]
                index += 1

                if(index<len(tokens) and tokens[index] == '='):
                    index += 1
                    index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
                    if(err):
                        return index, err, symbol_table
                    
        if(index<len(tokens) and tokens[index] != ';'):
            return index, "expected ; after an expression", symbol_table
        index +=1
        return index,None, symbol_table
    
    if(index<len(tokens) and tokens[index] == ","):
        while(index<len(tokens) and tokens[index] == ','):
            index += 1
            if(index<len(tokens) and not isIdentifier(tokens[index])):
                return index, "expected identifier"
            symbol_table[tokens[index]] = [dtype]
            index += 1

            if(index<len(tokens) and tokens[index] == '='):
                index += 1
                index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
                if(err):
                    return index, err, symbol_table
                
        if(index<len(tokens) and tokens[index] != ";"):
            return index, "expected ; ", symbol_table
        index += 1
        return index, None, symbol_table

    index, err, parameters = checkParametersList(tokens=tokens, index=index)

    if(err):
        return index, err, symbol_table
    
    # add function to the symbol table
    symbol_table[ident] = [dtype, parameters]

    if(index<len(tokens) and tokens[index] != ";"):
        return index, "Invalid function declaration", symbol_table
    
    return index+1, None, symbol_table

# checkForVariableDecl

In [352]:
# returns index, err, symbol_table
def checkForVariableDecl(tokens, index, symbol_table):
    
    dtype = ""
    isdec = False

    if(index<len(tokens) and tokens[index] == ';'):
        return index, None, symbol_table
    
    if(index<len(tokens) and tokens[index] in datatype):
        dtype = tokens[index]
        if(dtype == "void"):
            return index, "cannot be void", symbol_table
        isdec = True
        index += 1

    if(index<len(tokens) and not isIdentifier(tokens[index])):
        return index, "expected an identifier", symbol_table
    
    ident = tokens[index]
    if(isdec):
        symbol_table[ident] = [dtype]
    else:
        if(ident not in symbol_table):
            return index, "variable undeclared", symbol_table
    index += 1

    if(index<len(tokens) and tokens[index] == "="):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation"
        index += 1
        
        index,err,expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
        if(err):
            return index, err, symbol_table

    if(index<len(tokens) and tokens[index] == ","):
        while(index<len(tokens) and tokens[index] == ','):
            index += 1
            if(index<len(tokens) and not isIdentifier(tokens[index])):
                return index, "expected identifier"
            ident = tokens[index]

            if(isdec):
                symbol_table[ident] = [dtype]
            else:
                if(ident not in symbol_table):
                    return index, "variable undeclared", symbol_table
            
            index += 1

            if(index<len(tokens) and tokens[index] == '='):
                index += 1
                index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
                if(err):
                    return index, err, symbol_table
                
    return index, None, symbol_table


# Check Scope

In [353]:
# returns index, err
def checkScope(tokens, index, symbol_table, isfunc=False):

    

    if(index<len(tokens) and tokens[index] != '{'):
        return index, f"Expected {{ but found {tokens[index]}"
    index+=1

    local_symbol_table = {}
    local_symbol_table.update(symbol_table)

    while(index<len(tokens) and tokens[index] != "}"):
        

        if(index<len(tokens) and tokens[index] in datatype):
            
            index, err, sym_tab = checkFuncDecOrValDef(tokens=tokens, index=index, symbol_table=local_symbol_table)
            if(err):
                return index, err
            local_symbol_table.update(sym_tab)
            
            continue
            
        
        elif(index<len(tokens) and tokens[index] == "if"):
            
            index += 1
            if(index<len(tokens) and tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            index += 1
            index,err,expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table)
            if(err):
                return index, err
            if(index<len(tokens) and tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"

            index +=1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=local_symbol_table, isfunc=isfunc)
            if(err):
                return index, err
            
            #update else if

            if(index<len(tokens) and tokens[index] == "else"):
                index += 1
                index, err = checkScope(tokens=tokens, index=index, symbol_table=local_symbol_table, isfunc=isfunc)
                if(err):
                    return index, err
            
            continue

        elif(index<len(tokens) and tokens[index] == "while"):
            
            index += 1

            if(index<len(tokens) and tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            index+=1
            index,err,expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table)
            
            if(err):
                return index, err

            if(index<len(tokens) and tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"
            index +=1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=local_symbol_table, isfunc=isfunc)

            if(err):
                return index, err
            
            continue

        elif(index<len(tokens) and tokens[index] == "do"):
            
            index += 1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=local_symbol_table, isfunc=isfunc)
            if(err):
                return index, err

            if(index<len(tokens) and tokens[index] != "while"):
                return index, f"Expected while but found {tokens[index]}"
            index += 1

            if(index<len(tokens) and tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            index += 1
            index,err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table)
            
            if(err):
                return index, err

            if(index<len(tokens) and tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"
            index +=1

            if(index<len(tokens) and tokens[index] != ";"):
                return index, f"Expected ; but found {tokens[index]}"
            index +=1
            
            continue

        elif(index<len(tokens) and tokens[index] == "for"):
            
            index += 1
            if(index<len(tokens) and tokens[index] != "("):
                return index, "expected ("
            index += 1
            index, err, for_sym_tab = checkForVariableDecl(tokens=tokens, index=index, symbol_table=local_symbol_table)
            
            if(err):
                return index, err
            if(index<len(tokens) and tokens[index] != ";"):
                return index, "expected ;"
            index += 1
            index, err, expr1_dtype = checkExpr(tokens=tokens, index=index, symbol_table=for_sym_tab)
            if(err):
                return index, err
            if(index<len(tokens) and tokens[index] != ";"):
                return index, "expected ;"
            index += 1
            index, err, expr1_dtype = checkExpr(tokens=tokens, index=index, symbol_table=for_sym_tab)
            if(err):
                return index, err
            if(index<len(tokens) and tokens[index]!= ')'):
                return index, "expected )"
            index += 1
            index, err = checkScope(tokens=tokens, index=index, symbol_table=for_sym_tab, isfunc=isfunc)
            if(err):
                return index, err
            
            continue

        elif(isfunc and tokens[index] == "return"):
            if(index<len(tokens) and tokens[index] == "return"):
                index += 1
            
            index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table)
            if(err):
                return index, err

            if(index<len(tokens) and tokens[index] != ';'):
                return index, "expected ;"
            index += 1
            continue

        elif(index<len(tokens) and tokens[index] == "cin"):
            index += 1

            while(index<len(tokens) and tokens[index] == '>>'):
                index += 1
                if(not isIdentifier(tokens[index]) or not tokens[index] in local_symbol_table):
                    return index, "cin not applicable on non identifier"
                index += 1
            
            if(index<len(tokens) and tokens[index] != ';'):
                return index, "expected ;"
            index += 1
            continue

        elif(index<len(tokens) and tokens[index] == "cout"):
            index += 1
            while(index<len(tokens) and tokens[index] == '<<'):
                index += 1
                if(tokens[index] == "endl"):
                    index += 1
                else:
                    index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table)
                    if(err):
                        return index, err
            
            if(index<len(tokens) and tokens[index] != ';'):
                return index, "expected ;"
            index += 1
            continue

        else:
            
            index,err,expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table)
            
            
            if(err):
                return index, err
            if(index<len(tokens) and tokens[index] != ';'):
                return index, "unexpected token after an expression"
            index +=1
            
            continue
    
    return index+1,None
        

# Check Var Header

In [354]:
# returns index, err, dtype, ident

def checkVarHeader(tokens, index):
    # check construct of type <dtype> <ident>
    dtype=""
    ident=""

    if(index<len(tokens) and tokens[index] not in datatype):
        return index, "invalid data type", dtype, ident
    else:
        dtype = tokens[index]
    index += 1

    # check for identifier
    if(index<len(tokens) and not isIdentifier(tokens[index])):
        return index, "invalid identifier name", dtype, ident
    else:
        ident = tokens[index]
        
    return index+1, None, dtype, ident

# Check parameter list

In [355]:
# returns index, err, parameters

def checkParametersList(tokens, index):
    parameters = {}
    if(index<len(tokens) and tokens[index] != '('):
        return index, "missing opening parenthesis for a function", parameters
    
    index += 1
    if(index<len(tokens) and tokens[index] == ')'):
        return index+1, None, parameters
    
    while(True):
        index, err, dtype, ident = checkVarHeader(tokens, index)
        if(err):
            return index, err, parameters
        parameters[ident] = [dtype]
        if(index<len(tokens) and tokens[index] == ','):
            index += 1
        else:
            break
        
    
    if(index<len(tokens) and tokens[index] != ')'):
        return index, "missing closing parenthesis for a function", parameters
    
    return index+1, None, parameters


# check function definition and variable definition

In [356]:
# returns index, err
def checkFuncDefOrVarDef(tokens, index):
    index, err, dtype, ident = checkVarHeader(tokens=tokens, index=index)

    if(err):
        return index, err

    
    
    # check for multiple declaration
    
    global_symbol_table[ident] = [dtype]
    
    # variable decl
    if(index<len(tokens) and tokens[index] == ";"):
        if(dtype == "void"):
            return index, "wrong datatype for variable declaration"
        return index+1, None
    
    # variable def
    if(index<len(tokens) and tokens[index] == "="):
        if(dtype == "void"):
            return index, "wrong datatype for variable declaration"
        index += 1
        
        index,err,expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=global_symbol_table)
        return index,err

    if(index<len(tokens) and tokens[index] == ","):
        while(index<len(tokens) and tokens[index] == ','):
            index += 1
            if(index<len(tokens) and not isIdentifier(tokens[index])):
                return index, "expected identifier"
            global_symbol_table[tokens[index]] = [dtype]
            index += 1

            if(index<len(tokens) and tokens[index] == '='):
                index += 1
                index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=global_symbol_table)
                if(err):
                    return index, err
                
        if(index<len(tokens) and tokens[index] != ";"):
            return index, "expected ; "
        index += 1
        return index, None
                

    index, err, parameters = checkParametersList(tokens=tokens, index=index)
    

    if(err):
        return index, err

    # add function to the symbol table
    global_symbol_table[ident] = [dtype, parameters]

    if(index<len(tokens) and tokens[index] == ";"):
        return index+1, None
    
    local_symbol_table = {}
    local_symbol_table.update(global_symbol_table)
    local_symbol_table.update(parameters)

    index, err = checkScope(tokens, index, local_symbol_table, isfunc=True)

    #update
    if(err):
        return index,err

    return index, None
    

# Main function

In [357]:
i = 0
err = ""
while(i < len(tokens)):
    if(i<len(tokens) and tokens[i][0] == '#'):
        i,err = ignorePreprocessor(tokens=tokens, index=i)
        if(err):
            print(f"Error: error occured at token {i, tokens[i]}: {err}")
            break
        continue

    if(i<len(tokens) and tokens[i] == 'using'):
        i, err = ignoreTillSemiColon(tokens=tokens, index=i)
        if(err):
            print(f"Error: error occured at token {i, tokens[i]}: {err}")
            break
        continue
        
    if(i<len(tokens) and tokens[i] in datatype):
        # Def includes Decl, Def
        i, err = checkFuncDefOrVarDef(tokens=tokens, index = i)
        if(err and i<len(tokens)):
            print(f"Error: error occured at token {i, tokens[i]}: {err}")
            break
        continue
if(char_lit):
    print(f"Missing closing single quote after token ",i)
elif(const_str):
    print(f"Missing closing double quote after token ",i)
else:
    print("Programme parsed successfully")

Programme parsed successfully
